# Whereabouts Plans Template
This notebook will demonstrate how to create a whereabouts plan automatically.

<div style="text-align:center"><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/d3/Downtown_Austin_%28southward_view%2C_from_the_Capitol_Grounds_on_11th_street%29_%2823_November_2006%29.jpg/640px-Downtown_Austin_%28southward_view%2C_from_the_Capitol_Grounds_on_11th_street%29_%2823_November_2006%29.jpg" /></div>

## Introduction
The purpose of this notebook is to create a Street and Bridge Work Order plans based on segment IDs and additional comments on long line. Markings feature layers are published in the City of Austin ArcGIS Portal page available for public view as well. 

The data should already be available in the folder path as an excel spreadsheet. If the spreadsheet exists, a map document will be configured for spreadsheet use.

## Imports
The packages used for this project are:
- [pandas](https://pandas.pydata.org/) to create dataframe of extracted table and transform the data
- [pathlib](https://docs.python.org/3/library/pathlib.html) to find path to excel document if it exists
- [PyPDF2](https://pythonhosted.org/PyPDF2/) to merge PDFs for cover and pages in order
- [archook](https://github.com/JamesRamm/archook) to search for arcgis and makes arcpy available to python 
- [arcpy](https://pro.arcgis.com/en/pro-app/arcpy/get-started/what-is-arcpy-.htm) to create whereabouts markings plans using ESRI ArcMap Desktop software

In [1]:
import pandas as pd
from pathlib import Path
import PyPDF2

import archook 
archook.get_arcpy()
import arcpy

import datetime

## Constants

The date by month and day constant will determine the file pdf name to use as a dataframe. Folder path will determine where the plans will be created depending on the year. This is set to the top for the purpose of changing these constants as needed.

<i>The table below explains the purpose of each constant.</i>

| Constant | Description   |
|:--------:|----|
|<b>FOLDER</b>      |File directory used to import SBO whereabouts reports from email|
|<b>EXCEL_FILE</b>   |File directory name used to extact SBO whereabouts reports from file|
|<b>MXD</b>   |Map document used to create whereabouts plans template|
|<b>PDF_FILE</b>   |File directory name used to export pdf files|

In [8]:
%store -r FOLDER
%store -r EXCEL_FILE
MXD = arcpy.mapping.MapDocument(r"C:\Users\Govs\Projects\Whereabouts\Whereabouts_Template.mxd")
PDF_FILE = FOLDER[:-20] + r"\ATD_Whereabouts_"

## Methods
These functions will be used to extract and transform the data into a feasible format.

<i>The table below explains the purpose of each:</i>

| Method | Description   |
|:--------:|----|
|<b>df_text</b> |Changes element text to the pandas dataframe|
|<b>cover_aerial</b> |Zooms to selected features and extent for the dataframe|

In [4]:
# Formats element text to match pandas dataframe for cover
def df_text(r,row1,location,wg):
    date = datetime.datetime.now()
    for e in arcpy.mapping.ListLayoutElements(MXD,'TEXT_ELEMENT'):
            if e.name == 'SPECIFICATIONS':
                e.text = str(r["SPECIFICATIONS"])
            elif e.name == 'CREATED DATE':
                e.text =  "{}/{}/{}".format(date.month,date.day,date.year)
            elif e.name == 'REQUESTOR ID':
                e.text = str(r["Location ID"])
            elif e.name == 'row1':
                e.text = row1
            elif e.name == 'LOCATION':
                e.text = location
            elif e.name == 'author':
                e.text = 'Susanne Gov'
            elif e.name == 'WORK GROUPS':
                e.text = wg
            elif e.name == 'PAGE':
                e.text = str(r['PAGE'])
                
# Refreshes dataframe map aerial                
def aerial(layer,scale,select):
    dataframe = arcpy.mapping.ListDataFrames(MXD,"Layers")[0]
    dataframe.zoomToSelectedFeatures()
    dataframe.extent = layer.getSelectedExtent()
    dataframe.scale = dataframe.scale * scale
    if select == True:
        arcpy.SelectLayerByAttribute_management(layer,"CLEAR_SELECTION")
    arcpy.RefreshActiveView()

# Goes through mxd layers and allows selection/deselection based on query
def select_layers(q,dq,option):
    for l in arcpy.mapping.ListLayers(MXD,'TRANSPORTATION.markings*'):
        arcpy.SelectLayerByAttribute_management(l,option,q)
        l.definitionQuery = dq

# Creates pdf of MXD based on dataframe type
def create_pdf(df,scale,select):
    for index,row in df.iterrows():
        if 'Segment IDs' in df.columns:
            q = 'SEGMENT_ID IN({})'.format(str(str(row["Segment IDs"]).split(','))[1:-1])
            text1 = 'LOCATION'
            text2 = "{} from {} to {}".format(row["Street"],row["From"],row["To"])
            text3 = str(row['WORK GROUPS'])
            name = PDF_FILE + "Cover_{}.pdf".format(row["Location ID"])
        else:
            q = 'SEGMENT_ID = {}'.format(int(row["SEGMENT_ID"]))
            text1 = 'SEGMENT ID'
            text2 = str(row['SEGMENT_ID'])
            text3 = 'Markings'
            name = PDF_FILE + 'Page_{}_{}_{}.pdf'.format(row["Location ID"],row['SEGMENT_ID'],row['PAGE'])
        select_layers(q,q,'NEW_SELECTION')
        arcpy.SelectLayerByAttribute_management(layer,"NEW_SELECTION",q) 
        df_text(row,text1,text2,text3)
        select_layers(q,q,'CLEAR_SELECTION')
        aerial(layer,scale,select)
        arcpy.mapping.ExportToPDF(MXD,name,'PAGE_LAYOUT',640,480,200,'NORMAL','RGB',True,'DEFLATE','VECTORIZE_BITMAP')
        select_layers(q,'',"CLEAR_SELECTION")

## Import Datasets

In [5]:
if Path(EXCEL_FILE).exists():
    df = pd.read_excel(EXCEL_FILE,'Cover', index_col=0).reset_index()
    df1 = pd.read_excel(EXCEL_FILE,'Pages', index_col=0).reset_index()
    display(df1)

,Location ID,SEGMENT_ID,Comments,CROSSWALK,STOP_LINE,YIELD_LINE,Bicyclist symbol,Bike arrow,Diagonal crosshatch,Left arrow,Left/Through arrow,Only word,Right arrow,Shared lane (Sharrow) symbol,SPECIFICATIONS,PAGE
0,62963,2038719,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Install 1 stop line,2
1,62963,2038708,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Install 1 stop line,3
2,62963,3194305,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Install 1 stop line,4
3,SG-13247,3272912,"turn bays, lane lines, bike lanes",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,"Install turn bays, lane lines, bike lanes, 4 s...",2
4,SG-13247,3272907,"turn bays, lane lines, bike lanes",1.0,1.0,NaN,1.0,NaN,NaN,2.0,NaN,2.0,2.0,2.0,"Install turn bays, lane lines, bike lanes, 1 c...",3
5,SG-13247,3272914,"turn bays, lane lines, bike lanes",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,"Install turn bays, lane lines, bike lanes, 4 s...",4
6,SG-13247,3272909,"turn bays, lane lines, bike lanes",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,"Install turn bays, lane lines, bike lanes, 4 s...",5
7,SG-13247,3272915,"turn bays, lane lines, bike lanes",2.0,1.0,NaN,1.0,1.0,4.0,NaN,NaN,1.0,1.0,NaN,"Install turn bays, lane lines, bike lanes, 2 c...",6
8,SG-13247,3272910,"turn bays, lane lines, bike lanes",NaN,NaN,1.0,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,"Install turn bays, lane lines, bike lanes, 1 y...",7
9,SG-13247,2048238,"turn bays, lane lines, bike lanes",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8


# Creating the Map
A database connection to GISDM is needed to access the markings asset layers and the markings assets layers. We can open up a map document of the cover work orders and another map document of the pages work orders.

In [9]:
sde_path = r"Database Connections\GISDM.sde"

# Create Database connection 
arcpy.env.workspace = sde_path
if arcpy.Exists(sde_path) == False:
    arcpy.CreateDatabaseConnection_management("Database Connections","GISDM.sde", "ORACLE", 
                                              "sde:oracle11g:gisdm", "DATABASE_AUTH")

# Create street layer
layer = arcpy.mapping.ListLayers(MXD,"TRANSPORTATION.street_segment")[0]

## Creating Cover and Page PDFs

In [10]:
create_pdf(df,1,False)
create_pdf(df1,1.05,True)

## Merging Cover and Pages

In [11]:
for index,r in df.iterrows():
    pdf = PDF_FILE + "{}_Final.pdf".format(r['Location ID'])
    merger = PyPDF2.PdfFileMerger()
    merger.append("{}{}_{}.pdf".format(PDF_FILE,'Cover',r['Location ID']))
    for index,row in df1.iterrows():
        if r['Location ID'] == row['Location ID']:
            merger.append("{}{}_{}_{}_{}.pdf".format(PDF_FILE,'Page',row['Location ID'],row['SEGMENT_ID'],row['PAGE']))
    merger.write(pdf)
    merger.close()